In [1]:
import os

In [2]:
%pwd

'd:\\Courses\\Pau_ML_course\\Taxi_demand_predictor\\notebooks'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Courses\\Pau_ML_course\\Taxi_demand_predictor'

In [5]:
HOPSWORKS_PROJECT_NAME = 'taxi_demand_pred'

### Method to load .env file containing the HOPSWORKS API key

In [6]:
from src.paths import PARENT_DIR
from dotenv import load_dotenv

In [7]:
# load key-value pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR / ".env") 

HOPSWORKS_API_KEY = os.environ["HOPSWORKS_API_KEY"]

### Fetch Historical data for 2022 - 2023

In [8]:
from datetime import datetime
import pandas as pd
from src.data_processing import load_raw_data

In [9]:
from_year = 2022
to_year = 2023
print(f"Downloading raw data from {from_year} to {to_year}")

rides = pd.DataFrame()

for year in range(from_year, to_year + 1):
        
    # Download data for the whole year
    rides_one_year = load_raw_data(year)
    
    # appennd rows
    rides = pd.concat([rides, rides_one_year])

File for 2022-01 already exists in local directory
File for 2022-02 already exists in local directory
File for 2022-03 already exists in local directory
File for 2022-04 already exists in local directory
File for 2022-05 already exists in local directory
File for 2022-06 already exists in local directory
File for 2022-07 already exists in local directory
File for 2022-08 already exists in local directory
File for 2022-09 already exists in local directory
File for 2022-10 already exists in local directory
File for 2022-11 already exists in local directory
File for 2022-12 already exists in local directory
File for 2023-01 already exists in local directory
File for 2023-02 already exists in local directory
File for 2023-03 already exists in local directory
2023-04 file is not available
2023-05 file is not available
2023-06 file is not available
2023-07 file is not available
2023-08 file is not available
2023-09 file is not available
2023-10 file is not available
2023-11 file is not avail

Check total rows in the data

In [10]:
print(f"Raw data size of both 20222 and 220223 year: {len(rides)}")

Raw data size of both 20222 and 220223 year: 49038829


### Transform the raw data into time series data

In [11]:
from src.data_processing import transform_raw_data_into_ts_data

In [12]:
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 262/262 [00:22<00:00, 11.49it/s]


### Save the data in feature store

- Following are the steps

In [13]:
import hopsworks

#### Connect to project on hopsworks

In [14]:
project = hopsworks.login(
    project = HOPSWORKS_PROJECT_NAME,
    api_key_value = HOPSWORKS_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/59568


#### Connect to the feature store

In [15]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


#### Now, save the data into the feature store

- to save data, we need to use feature groups which is the API for writing data into feature store

- To use feature group, we need to set it's name and version

In [16]:
FEATURE_GROUP_NAME = "time_series_hourly_feature_group"
FEATURE_GROUP_VERSION = 1

#### Now, create the feature group

In [18]:
feature_group = feature_store.get_or_create_feature_group(
    name = FEATURE_GROUP_NAME,
    version = FEATURE_GROUP_VERSION,
    description = "Time series data at hourly frequency",
    primary_key = ["pickup_location_id", "pickup_hour"],
    event_time = "pickup_hour"
)

#### Finally save the data into the feature store

In [19]:
feature_group.insert(ts_data, write_options = {"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/59568/fs/58464/fg/61982


Uploading Dataframe: 0.00% |          | Rows 0/2861040 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/59568/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x285024b0d90>, None)